In [1]:
import pandas as pd
import numpy as np
import os

### 2018年6月 - 2021年6月基金日净值数据

In [2]:
# 按文件夹搜索导入数据
mydatalist = []
for info in os.listdir('基金数据210630/180630/'):
    domain = os.path.abspath('基金数据210630/180630/')
    info = os.path.join(domain,info)
    data = pd.read_csv(info)
    mydatalist.append(data)

nav_data = pd.concat(mydatalist)

#### 基金主体信息合并

In [3]:
maininfo = pd.read_csv('基金数据210630/FUND_MainInfo.csv') # 仅保留了契约开放式的股票型&混合型基金

In [4]:
maininfo.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2404 entries, 0 to 2403
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FundID           2404 non-null   int64  
 1   MasterFundCode   2404 non-null   int64  
 2   FullName         2404 non-null   object 
 3   FundCompanyID    2404 non-null   int64  
 4   FundCompanyName  2404 non-null   object 
 5   ManagementFee    2404 non-null   float64
 6   InceptionDate    2404 non-null   object 
 7   FundTypeID       2404 non-null   object 
 8   FundType         2404 non-null   object 
 9   CategoryID       2404 non-null   object 
 10  Category         2404 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 206.7+ KB


In [5]:
unitinfo = pd.read_csv('基金数据210630/FUND_UnitClassInfo.csv') # 仅保留了所有非定期开放式基金

In [6]:
unitinfo.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13039 entries, 0 to 13038
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FundID           13039 non-null  int64  
 1   MasterFundCode   12991 non-null  float64
 2   FundClassID      13039 non-null  float64
 3   Symbol           13039 non-null  int64  
 4   Expired          13039 non-null  int64  
 5   ShortName        13039 non-null  object 
 6   InceptionDate    13039 non-null  object 
 7   Cycle            0 non-null      float64
 8   RedeemStartDate  11617 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 916.9+ KB


In [7]:
print(maininfo.shape, unitinfo.shape)

(2404, 11) (13039, 9)


In [8]:
print(len(maininfo.FundID.unique()),len(maininfo.MasterFundCode.unique()))

2404 2404


In [9]:
print(len(unitinfo.FundID.unique()),len(unitinfo.MasterFundCode.unique()))

7929 7929


In [10]:
fundsinfo = pd.merge(maininfo,unitinfo, how = 'inner', on = 'FundID')
print(fundsinfo.shape)

(2254, 19)


In [11]:
fundsinfo.head(1)
fundsinfo = fundsinfo.drop(columns =['MasterFundCode_x','MasterFundCode_y','InceptionDate_x','Expired','Cycle'])

In [12]:
print(len(fundsinfo.Symbol.unique()),len(fundsinfo.FundClassID.unique()))

2237 2237


#### 基金日净值与主体信息合并

对比合并整理后的基金主体信息与日净值表格中的索引列’基金代码‘、’基金份额类别ID’，发现单个基金代码及份额类别ID存在对应多支基金主体信息的情况，但这类索引列在日净值表上对应的净值为单日单个，意味着不同主体信息对应同一索引ID的多支基金存在主要基金与连接基金的关系，其净值数据实为一支基金的净值，对于这类重复索引值的基金，作仅保留起始日期较早一支的处理。

In [16]:
fundsinfo1 = fundsinfo[['FundID','FundCompanyID','ManagementFee',/
                        'Symbol','ShortName','InceptionDate_y',/
                        'RedeemStartDate']].drop_duplicates(subset=['Symbol'], keep='first') # 按索引值基金代码去重

SyntaxError: invalid syntax (<ipython-input-16-eb53929fa1ab>, line 1)

In [14]:
nav_data1 = pd.merge(fundsinfo1, nav_data, how='inner', on='Symbol')
print(nav_data1.shape)

NameError: name 'fundsinfo1' is not defined

In [122]:
# 将可赎回起始日中空值用其份额成立日补齐
nav_data1.loc[nav_data1['RedeemStartDate'].isnull(),'RedeemStartDate']= nav_data1[nav_data1['RedeemStartDate'].isnull()]['InceptionDate_y']
# Check Output
len(nav_data1[nav_data1.RedeemStartDate.isna()])

0

In [124]:
dt = nav_data1.drop(columns=['Symbol','InceptionDate_y','FundClassID']) # 去掉多余列

In [126]:
dt.head(5)

,FundID,FundCompanyID,ManagementFee,ShortName,RedeemStartDate,TradingDate,NAV,AccumulativeNAV
0,108426,1059,1.5,华夏成长混合,2002/1/30,2018/6/30,1.045,3.456
1,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/2,1.032,3.443
2,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/3,1.023,3.434
3,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/4,1.011,3.422
4,108426,1059,1.5,华夏成长混合,2002/1/30,2018/7/5,0.995,3.406


In [ ]:
dt.to_csv('data1806_2106.csv',encoding='utf-8-sig')